# Datenaustausch WIAG/FactGrid, Teil 2
## Formatiere CSV-Daten aus WIAG als V1-Statements für QuickSteps  
siehe [Einführung in QuickStatements](https://www.wikidata.org/wiki/Help:QuickStatements)
und die [Properties des FactGrid](https://database.factgrid.de/wiki/FactGrid:Directory_of_Properties)

Änderungen

- *2023-12-11* Verwende die Gruppe des höchsten erreichten Amtes in der englischen Beschreibung und für P165

Inhalt:

- [neue Objekte in FactGrid anlegen](#neue-Objekte-in-FactGrid-anlegen)

In [ ]:
using CSV, DataFrames, Dates

In [ ]:
ENV["COLUMNS"] = 120

In [ ]:
input_path = "C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

In [ ]:
output_path = input_path

## neue Objekte in FactGrid anlegen
Es wird davon ausgegangen, dass die betreffenden Personen noch nicht in FactGrid vorhanden sind. Die Spalte "FactGrid_ID" sollte also für alle Einträge leer sein.

Daten aus WIAG einlesen: Quelle Export "CSV Personendaten"

In [ ]:
input_file = "WIAG-can-Bamberg.csv"
input_path_file = joinpath(input_path, input_file)

In [ ]:
df_person_all = CSV.read(input_path_file, DataFrame);
nrow(df_person_all)

Betrachte nur Neu-Zugänge (keine FactGrid-ID)

In [ ]:
df_person_in = subset(df_person_all, :FactGrid_ID => ByRow(ismissing));
nrow(df_person_in)

Lies die Amtsdaten ein (zunächst für P165). Sie werden auch verwendet, um eine Beschreibung zusammenzustellen.

In [ ]:
input_file = "WIAG-can-roles-Bamberg.csv"
input_path_file = joinpath(input_path, input_file)

In [ ]:
df_role_all = CSV.read(input_path_file, DataFrame);
nrow(df_role_all)

Definiere Funktionen:  
Lege die zwei Rollen/Amtszeiten fest, die für die Beschreibung verwendet werden.  
Priorisiere nach Gruppe der Rolle (siehe `role_group_rank_list`) und dann nach Amtszeit (jüngste zuerst)

In [ ]:
function describe_role(role)
    inst_or_dioc = !ismissing(role[:institution]) ? role[:institution] : role[:diocese]    
    
    date_info = ""
    if !ismissing(role[:date_begin]) && !ismissing(role[:date_end])
        date_info = role[:date_begin] * "-" * role[:date_end]
    elseif !ismissing(role[:date_begin])
        date_info = role[:date_begin]
    elseif !ismissing(role[:date_end])
        date_info = "bis " * role[:date_end]
    end
    
    description = role[:name]
    if !ismissing(inst_or_dioc)
        description *= " " * inst_or_dioc
    end
    if date_info != ""
        description *= " " * date_info
    end
    
    return description    
end

In [ ]:
role_group_rank_list = [
    "Q648236", # Leiter (Erz-)diözese (Altes Reich)
    "Q648232", # Domdignitär Altes Reich            
    "Q648226", # Domkleriker Altes Reich
    "Q648233", # Klosterangehöriger mit Leitungsamt
];

Häufigkeit der priorisierten Gruppen

In [ ]:
for rg in role_group_rank_list
    println(rg, ": ", count(isequal(rg), df_role_all.role_group_fq_id))
end

In [ ]:
"""
    lt_role_group_fq_id(a, b, role_group_rank_list)

compare `a` and `b` based on `role_group_rank_list`
"""
function less_than_role_group_fq_id(a, b, role_group_rank_list)
    if ismissing(a)
        return false
    end
    if ismissing(b)
        return true
    end
    
    lt = false
    for rg_prio in role_group_rank_list
        # println(rg_prio, " a: ", a, " b: ", b)
        if a == rg_prio && b != rg_prio
            lt = true
            break
        end
        if a != rg_prio && b == rg_prio
            lt = false
            break
        end
    end
    
    return lt        
end

lt_rg_fq_id(a, b) = less_than_role_group_fq_id(a, b, role_group_rank_list)

In [ ]:
function description_by_role(df)
    N_ROLE_4_DESCRIPTION = 2
    df_s = sort(df, [:date_sort_key], rev = true)
    df_s = sort(df_s, [:role_group_fq_id], lt = lt_rg_fq_id, alg = MergeSort)
    description_list = String[]
    for row in eachrow(df_s)
        push!(description_list, describe_role(row))
    end
    head_list = first(unique(description_list), N_ROLE_4_DESCRIPTION)
    return join(head_list, ", ")
end

In [ ]:
dfg_role_all = groupby(df_role_all, [:person_id]);
df_role_description = combine(dfg_role_all, description_by_role);
nrow(df_role_description)

In [ ]:
df_role_description[200:205, :]

Führe die Beschreibungen mit den übrigen Personendaten zusammen. (join)

In [ ]:
columns = [
    :person_id => :person_id,
    :x1 => :summary_roles,
]

df_person = innerjoin(df_person_in, select(df_role_description, columns), on = :id => :person_id);
nrow(df_person)

Für die englische Beschreibung wird die Bezeichnung der am höchsten priorisierten Ämtergruppe verwendet

In [ ]:
function description_by_role_en(df)
    N_ROLE_4_DESCRIPTION = 1
    df_s = sort(df, [:date_sort_key], rev = true)
    df_s = sort(df_s, [:role_group_fq_id], lt = lt_rg_fq_id, alg = MergeSort)
    if nrow(df_s) < 1
        return missing
    else
        return first(df_s)[:role_group_en]
    end
end

In [ ]:
df_role_description_en = combine(dfg_role_all, description_by_role_en);
nrow(df_role_description_en)

In [ ]:
columns = [
    :person_id => :person_id,
    :x1 => :best_role_group_en,
]

In [ ]:
df_person = innerjoin(df_person, select(df_role_description_en, columns), on = :id => :person_id);
nrow(df_person)

In [ ]:
sort!(df_person, [:corpus, :id]);

In [ ]:
df_person[1:4, [:id, :displayname, :summary_roles, :best_role_group_en]]

Wähle für die Liste der Rollen die relevanten Spalten aus.  
*2023-12-18* zunächst werden die Amtsdaten noch nicht detailliert im FactGrid eingetragen. Die Personen werden aber schon Rollengruppen zugeordnet.

In [ ]:
columns = [
    :person_id => :person_id,
    :role_group_fq_id => :P165
]

In [ ]:
df_role = select(df_role_all, columns);

Unter der Property P165 werden die Gruppen der Ämter eingetragen. Relevante Gruppen werdne hier über entsprechenden FactGrid-ID ausgewählt.  
*2023-12-19* aktuell werden alle Gruppen verwendet

In [ ]:
relevant_role_group_fq_id = [
    "Q254893",
    "Q385344",
    "Q648226",
    "Q648227",
    "Q648228",
    "Q648229",
    "Q648230",
    "Q648232",
    "Q648233",
    "Q648234",
    "Q648235",
    "Q648236",
    "Q648239",
];

In [ ]:
is_relevant(s) = !ismissing(s) && s in relevant_role_group_fq_id

In [ ]:
df_role_group = subset(df_role, :P165 => ByRow(is_relevant));
nrow(df_role_group)

Doppeleinträge sind nicht sinnvoll. Lösche doppelte Einträge.

In [ ]:
df_role_group = unique(df_role_group);
nrow(df_role_group)

Das deutsche Beschreibungsfeld soll Lebensdaten mit der Zusammenfassung der Amtsdaten enthalten.

In [ ]:
join_komma(a, b) = join((a, b), ", ")
transform!(df_person, [:biographical_dates, :summary_roles] => ByRow(join_komma) => :description_de);

Das englische Beschreibungsfeld soll Lebensdaten mit der am höchsten priorisierten Ämtergruppe enthalten. Falls der Name der Gruppe nicht übersetzt ist oder falls es keine Ämter gibt, wird für die Beschreibung 'missing' ausgegeben.

In [ ]:
transform!(df_person, [:biographical_dates, :best_role_group_en] => ByRow(join_komma) => :description_en);

Das FactGrid arbeitet für Wikipedia nicht mit URL-codierten Adressen.
*2023-12-13* WIAG gibt direkt unkodierte URLs aus, daher sind die folgenden Schritte auskommentiert

In [ ]:
# unescape_not_mg(s) = ismissing(s) ? s : URIs.unescapeuri(s)

In [ ]:
# transform!(df_person, :Wikipedia => ByRow(unescape_not_mg) => :Wikipedia);

In [ ]:
columns = [:givenname, :prefix, :familyname, :Wikipedia]
dropmissing(df_person, :Wikipedia)[5:9, columns]

Benenne die Spalten um entsprechend den Konventionen des FactGrid.

In [ ]:
columns = [
    :displayname => :Lde,
    :description_de => :Dde,
    :description_en => :Den,
    :date_of_birth => :P77,
    :date_of_death => :P38,
    :GND_ID => :P76,
    :GSN => :P472,
    :id => :P601,
    :Wikidata_ID => :Swikidatawiki,
    :Wikipedia => :Sdewiki
]

rename!(df_person, columns);

Kopiere das Label in Deutsch für die anderen Sprachen

In [ ]:
df_person.Len = df_person.Lde;
df_person.Lfr = df_person.Lde;
df_person.Les = df_person.Lde;

In [ ]:
df_person[111:114, [:P601, :givenname, :prefix, :familyname, :Len, :Den]]

Füge Daten ein, die für alle Personen gleich sind:
Mensch, Teil der Germania Sacra Forschungsdaten, männlich

In [ ]:
insertcols!(df_person, 
    :P2 => "Q7",
    :P131 => "Q153178",
    :P154 => "Q18"
);    

Definiere Umwandlungsfunktionen  
*offen:* Datumsangaben sind noch zu besprechen, wegen der Verarbeitung von unscharften Zeitangaben.

In [ ]:
fqs_string(s) = "\"" * string(s) * "\""

In [ ]:
function create(out, row, col_list) 
    println(out, "CREATE")    
    for col in col_list
        println(out, "LAST\t", string(col), "\t", fqs_string(row[col]))        
    end
end

In [ ]:
"""
    set_property_list (out, row, object, col_list, fmt_list)

    

# Arguments
- `out::IOStream`: output stream
- `row::DataFrameRow`: input data
- `object`: FactGrid identifier or "LAST"
- `col_list`: list of colums where data should be read from
- `fmr_list`: list of formatting functions (one for each column)
"""
function set_property_list(out, row, object, col_list, fmt_list)
    for (col, fmt) in zip(col_list, fmt_list)
        if (!ismissing(row[col]))
            println(out, object, "\t", string(col), "\t", fmt(row[col]))
        end
    end
end

Funktionsdefinition: Lies die Amtsdaten für einen bestimmten Domherren

In [ ]:
"""
    set_role(df_role, id)

extract roles for person with `id`
"""
function set_role(out, df_role, id, object, property)
    property = Symbol(property)
    df_pr = subset(df_role, :person_id => ByRow(isequal(id)));
    df_pr = subset(df_pr, )
    fq_id_list = unique(df_pr[!, property])
    for fq_id in fq_id_list
        println(out, object, "\t", property, "\t", fq_id)
    end
    return nothing
end


Gib ausgewählte Elemente aus `df_person` aus. Falls es schon eine Datei mit gleichem Namen im angegebenen Verzeichnis gibt, wird die Datei überschrieben.

In [ ]:
date_key = Dates.format(now(), "yyyy-mm-dd")

In [ ]:
output_file = "Insert_WIAG-Bamberg_" * date_key * "_FQ.csv"
output_path_file = joinpath(output_path, output_file)

In [ ]:
open(output_path_file, "w") do out_stream
    for row in eachrow(df_person)
        create(out_stream, row, [:Lde, :Len, :Lfr, :Les, :Dde, :Den])
        set_property_list(out_stream, row, "LAST", 
            [:P2, :P131, :P154, :P601, :P76, :P472, :Swikidatawiki, :Sdewiki], 
            [identity, identity, identity, fqs_string, fqs_string, fqs_string, fqs_string, fqs_string])
        set_role(out_stream, df_role_group, row[:P601], "LAST", :P165)
    end        
end